# Customer Clustering #01.SQL

SQLで、オリジナルデータの概要を理解し、今回の分析に不要なデータを削ぎ落とす作業（行数を1,000万から14万へ）

In [1]:
import pandas as pd
import sqlite3
import pandas.io.sql as psql

## データ読み込み

In [2]:
train_data = pd.read_csv("train.csv")

##### train_dataのレコード数は「9,917,530」

In [3]:
print('Number of Rows: %i   Number of Columns: %i' % train_data.shape)

Number of Rows: 9917530   Number of Columns: 54


## データ理解＆前処理

In [ ]:
前処理（SQLで必要なレコードに絞る）

### expedia.dbにtrain_dataを格納

In [4]:
dbn = "expedia.db"
con = sqlite3.connect(dbn)

In [5]:
train_data.to_sql("data", con, if_exists="replace")

##### (1) srch_id（検索条件の件数）は「399,344」

In [6]:
your_sql = """
    SELECT COUNT(Distinct(srch_id)) as srch_id_cnt
    FROM data
    LIMIT 10
"""

In [7]:
psql.read_sql(con=con, sql=your_sql)

,srch_id_cnt
0,399344


In [ ]:
(SELECT srch_id, SUM(booking_bool) FROM data GROUP BY srch_id) AS srch_booking_bool

##### (2) (1)のうち、購入あり（booking_bool = 1）のsrch_id（検索条件の件数）は「276,593」

In [191]:
your_sql = """
    SELECT
        b.booking_bool,
        COUNT(Distinct(a.srch_id)) as srch_id_cnt,
        (1.0*COUNT(Distinct(a.srch_id)))/
            (SELECT COUNT(Distinct(srch_id)) as srch_id_cnt FROM data) as rate
    FROM data AS a
    INNER JOIN (SELECT srch_id, SUM(booking_bool) AS booking_bool FROM data GROUP BY srch_id) AS b 
        ON a.srch_id = b.srch_id
    GROUP BY b.booking_bool
    ORDER BY srch_id_cnt DESC
    LIMIT 10
"""

In [192]:
psql.read_sql(con=con, sql=your_sql)

,booking_bool,srch_id_cnt,rate
0,1,276593,0.692618
1,0,122751,0.307382


##### (3) (2)のうち、Expedia.com（site = 5）のsrch_idの数（検索条件の件数）は「173,575」

In [65]:
your_sql = """
    SELECT
        CASE 
            WHEN site_id = 5 THEN "Expedia.com"
            ELSE "Others"
        END AS site_name,
        COUNT(Distinct(srch_id)) as srch_id_cnt,
        (1.0*COUNT(Distinct(srch_id))/
            (SELECT COUNT(Distinct(srch_id)) as srch_id_cnt FROM data WHERE booking_bool = 1)) as rate
    FROM data
    WHERE booking_bool = 1
    GROUP BY
        CASE 
            WHEN site_id = 5 THEN "Expedia.com"
            ELSE "Other"
        END
    ORDER BY srch_id_cnt DESC
    LIMIT 10
"""

In [66]:
psql.read_sql(con=con, sql=your_sql)

,site_name,srch_id_cnt,rate
0,Expedia.com,173575,0.627547
1,Others,103018,0.372453


##### (4) (3)のうち、居住地がUSA（visitor_location_country_id = 219）のsrch_idの数（検索条件の件数）は「158,464」

In [193]:
your_sql = """
    SELECT
        CASE 
            WHEN visitor_location_country_id = 219 THEN "USA"
            ELSE "Other"
        END AS visitor_location_country_name,
        COUNT(Distinct(srch_id)) as srch_id_cnt,
        (1.0*COUNT(Distinct(srch_id))/
            (SELECT COUNT(Distinct(srch_id)) as srch_id_cnt 
            FROM data WHERE booking_bool = 1 AND site_id = 5)) as rate
    FROM data
    WHERE booking_bool = 1
        AND site_id = 5
    GROUP BY
        CASE 
            WHEN visitor_location_country_id = 219 THEN "USA"
            ELSE "Other"
        END
    ORDER BY srch_id_cnt DESC
    LIMIT 10
"""

In [194]:
psql.read_sql(con=con, sql=your_sql)

,visitor_location_country_name,srch_id_cnt,rate
0,USA,158464,0.912943
1,Other,15111,0.087057


##### (5) (4)のうち、目的地がUSA（prop_country_id =219）のsrch_idの数（検索条件の件数）は「139,302」

In [220]:
your_sql = """
    SELECT
        CASE 
            WHEN prop_country_id = 219 THEN "USA"
            ELSE "Other"
        END AS prop_country_name,
        COUNT(Distinct(srch_id)) as srch_id_cnt,
        (1.0*COUNT(Distinct(srch_id))/
            (SELECT COUNT(Distinct(srch_id)) as srch_id_cnt 
            FROM data WHERE booking_bool = 1 
            AND site_id = 5 AND visitor_location_country_id = 219)) as rate
    FROM data
    WHERE booking_bool = 1
        AND site_id = 5
        AND visitor_location_country_id = 219
    GROUP BY
        CASE 
            WHEN prop_country_id = 219 THEN "USA"
            ELSE "Other"
        END
    ORDER BY srch_id_cnt DESC
    LIMIT 10
"""

In [221]:
psql.read_sql(con=con, sql=your_sql)

,prop_country_name,srch_id_cnt,rate
0,USA,139302,0.879077
1,Other,19162,0.120923


### 分析対象データ

##### 今回は「Expedia.comでアメリカ合衆国の国内旅行の購入ログ」を分析対象とする（詳細条件を以下の通り）

・購入有無：購入あり（booking_bool = 1）<br>
・利用サイト：Expedia.com（site_id = 5）<br>
・居住地：USA（visitor_location_country_id = 219）<br>
・目的地：USA（prop_country_id = 219）<br>

##### 分析対象データをexpedia.dbから抽出しpandasに格納

In [225]:
your_sql = """
    SELECT *
    FROM data
    WHERE booking_bool = 1
        AND site_id = 5
        AND visitor_location_country_id = 219
        AND prop_country_id = 219
"""

In [226]:
book_data = psql.read_sql(con=con, sql=your_sql)

In [227]:
book_data.head()

,index,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,68,8,2013-03-20 17:50:44,5,219,NaN,NaN,219,27669,3,...,None,NaN,NaN,NaN,0.0,0.0,NaN,1,96.41,1
1,211,25,2012-12-11 13:03:42,5,219,NaN,NaN,219,23228,4,...,None,NaN,NaN,NaN,0.0,0.0,NaN,1,47.10,1
2,243,28,2012-11-26 10:13:17,5,219,NaN,NaN,219,13996,4,...,None,NaN,NaN,NaN,-1.0,0.0,9.0,1,286.32,1
3,387,39,2013-04-06 18:08:58,5,219,NaN,NaN,219,109855,2,...,None,NaN,NaN,NaN,0.0,0.0,NaN,1,60.77,1
4,653,53,2013-02-13 07:00:24,5,219,NaN,NaN,219,101178,4,...,None,NaN,NaN,NaN,-1.0,0.0,17.0,1,347.65,1


In [228]:
print('Number of Rows: %i   Number of Columns: %i' % book_data.shape)

Number of Rows: 139302   Number of Columns: 55
